In [10]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium

# Problem 4

Create a choropleth showing the number of burglaries per 1000 residents for each census tract.

In [11]:
tn = gpd.read_file('../data/tn.geojson')
tn_shape = gpd.read_file("../data/tl_2019_47_tract.shp").to_crs(crs = tn.crs)
tn_shape.columns = tn_shape.columns.str.lower()
tn_shape = (tn_shape.rename(columns = {'statefp':'state',
                            'countyfp':'county',
                            'namelsad':'tract_name',
                            'tractce':'tract'}
                )
                #.drop(columns = ['name', 'namelsad'])
            )

In [12]:
tn['burglary_count'] = tn.groupby('tract')['tract'].transform('count')

In [13]:
tn['burgs_per_1000'] = (tn['burglary_count'] * 1000) / tn['population']

In [14]:
tnt = tn[tn['burgs_per_1000'] != np.inf]
droplist = ['state', 'name', 'state_name', 'tract_name', 'mtfcc',
           'funcstat', 'index_right', 'rpa', 'intptlat', 
           'intptlon', 'location_code',
           'report_type', 'incident_reported', 'weapon_description',
           'weapon_primary', 'report_type_description',
           'incident_status_code', 'incident_status_description', 
           'primary_key', 'geoid']
tnt = tnt.drop(columns = droplist)
tnt['aland'] = tnt['aland']/2589988
tnt['population_density'] = tnt['population'] / tnt['aland']

In [15]:
tnt.to_file('../data/tnt.geojson', index = False, driver = 'GeoJSON')

In [16]:
centroid163 = tn_shape.loc[tn_shape['tract'] == '016300', 'geometry'].centroid

<ipython-input-16-4a97647ee8ad>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid163 = tn_shape.loc[tn_shape['tract'] == '016300', 'geometry'].centroid


In [ ]:
m = folium.Map(location=[centroid163.y,centroid163.x], zoom_start=12)

folium.Choropleth(
    geo_data=tnt,
    name="choropleth",
    data=tnt,
    columns=["tract","burgs_per_1000"],
    key_on="feature.properties.tract",
    fill_color="Spectral",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Burglaries Per 1000 People",
).add_to(m)

folium.LayerControl().add_to(m)

m